# 한국어 뉴스 기사분류

- BalancedNewsCorpus_train.csv, BalancedNewsCorpus_test.csv는 국어원 뉴스 자료에서 9개 분야의 신문별 균형을 맞춘 자료로, 학습용 9,000개 시험용 1800 자료가 있는 파일이다.
- 이 파일을 가지고 https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라 한국어 뉴스기사 분류기를 만들어라
- training data에서 evaluation data를 나누어 사용할 수 있다.(필요시)
- 화일 이름은 MidTermProject_학번_이름
- 마감: 2022년 10월 27일 목요일 오후 11시 59분 59초까지!

## 목표

- csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
(Field, Iterator, train,test, evaluation and prediction)
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기 

- 외부에서 학습된 한국어 단어 임베딩을 torchtext에 통합하여 사용하기 (word2vec, glove, fasttext 중 골라서 사용)
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 기사들의 분류 결과를 보이기

- 참고 사이트
    - https://pytorch.org/text/
    - http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    - https://github.com/pytorch/text
    - https://mc.ai/using-fine-tuned-gensim-word2vec-embeddings-with-torchtext-and-pytorch/

## 이 자료를 위해 사전학습된 임베딩
- 필요에 따라 선택하여 사용할 수 있음

### Word2vec 모델
 - 노트 : https://drive.google.com/file/d/1KOv901TPv5gepEdd4cCsJWoCHVaAV-A-/view?usp=sharing
 - Word2Vec 형태소 모델 : https://drive.google.com/file/d/1DDx6lRSTVULRFP3kslQLZsuoGZJOtoR1/view?usp=sharing
 - Word2Vec 어절 모델 : https://drive.google.com/file/d/1-RuEk-MhULduAbizgt3wjsMOCL3sM_pl/view?usp=sharing

- from gensim.models.keyedvectors import KeyedVectors
- Word2Vec_300D_space_model = KeyedVectors.load_word2vec_format(path + 'Word2Vec_300D_space.model', binary=False, encoding='utf-8')


### Faxttext 모델

- fasttext 형태소 모델: https://drive.google.com/file/d/1-EBaAtFK7chB6qqckKmLdghR62SebEYK/view?usp=sharing
- fasttext 어절 모델: https://drive.google.com/file/d/1-0D7Fe5oG_z9pQqOjkewtsuV_uVkh_dF/view?usp=sharing
- fasttext 형태소 자모 모델: https://drive.google.com/file/d/1-WW_qWQZ2q3Jj9fXXex82dYHWIMHGVri/view?usp=sharing
- fasttext 어절 자모 모델: https://drive.google.com/file/d/1-P2b8Dp09fZYO2Y__wjPNmS77PF7kfqV/view?usp=sharing


- from gensim.models.keyedvectors import KeyedVectors
- fasttext_model2 = KeyedVectors.load_word2vec_format(path + 'fasttext_morph_300.model', binary=False, encoding='utf-8')

## Glove 모델

- https://drive.google.com/drive/folders/1pzVO0jwx1Zf8p4hjf4JQn81XWzktsIdg?usp=sharing 


- from gensim.models.keyedvectors import KeyedVectors
- Glove_model = KeyedVectors.load_word2vec_format(# 모델 경로 , binary=False, encoding='utf-8')


## 정리
### 설계 요약
FastText 형태소 모델을 사용하였으며, 불용어 제거 및 문단 구분 태그 `<p>`와 `</p>`를 제거하는 전처리를 진행하였다.
이러한 선택은 Word2Vec에 비해 FastText의 분류 성능이나 강건성 측면에서 나으며, 어절 토큰보다 형태소 토큰을 기반으로 하였을 때 분류 성능이 높다는 기존 연구 결과[1]를 참고하여 진행하였다.

학습은 CNN으로 하였고, 2, 3, 4 높이의 필터를 각각 100개씩 사용하여 총 300개의 n-gram을 사용한 효과를 내었다.
11번째 epoch에서 최소 validation loss를 달성하였고, 테스트 데이터에 대해서 loss는 0.575, accuracy는 80.06%가 나왔다.

[1] 박서윤, 강예지, 강혜린, 장연지, 김한샘, 관용표현 중의성 해소를 위한 다층위 임베딩 연구, 제33회 한글 및 한국어 정보처리 학술대회 논문집, 2021

### 성능 및 평가
| 예상      | 답안      | 정답    |
|-----------|-----------|---------|
| 스포츠    | 생활      | X       |
| 정치      | 정치      | O       |
| 사회      | 경제      | X       |
| 문화      | 문화      | O       |
| 생활      | 사회      | X       |
| 스포츠    | 스포츠    | O       |
| 문화      | 연예      | X       |
| 미용/사회 | 미용/사회 | O       |
| IT/과학   | IT/과학   | O       |
|           |           | 5/9=56% |
        
참고로, 무작위 예측기가 이보다 성능이 좋을 확률은
$$\sum_{k=5}^{9} {9 \choose k} \left(\frac{1}{9}\right)^k \left(\frac{8}{9}\right)^{9-k} = 1.45\%$$
이다.

나아가, 본인도 해당 문장들 중 두 개를 잘 못 분류할 정도로 까다로운 경우가 존재하였다.
예컨대 "정부는 2012년 예산의 공고안과 배정계획을 1월3일 국무회의에서 의결하고 연초부터 바로 집행에 들어간다.
세계 경제의 불확실성이 높아지고 경기가 둔화할 가능성이 높은 만큼 조기 집행에 박차를 가할 예정이다"은 경제라고 생각하였지만 답은 정치였고, 놀랍게도 학습된 모델은 이를 정확히 예측하였다.
모델이 틀린 답을 내놓은 문장들을 보면 실제로는 "생활" 분류의 기사이지만 "국가 대표팀"이 등장하는 등 실제로 어려운 경우에 해당하였다.

In [79]:
# !pip install --upgrade pip
# !pip install pandas
# !pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install torchtext==0.9.0
# !pip install ipywidgets
# !pip install gensim

In [80]:
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# !bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

In [81]:
PATH = "."  # "./drive/MyDrive"

In [82]:
import pandas as pd
import csv

data_set = f"{PATH}/BalancedNewsCorpus_" # test.csv / train.csv
test_df, train_df = (pd.read_csv(f"{data_set}{x}.csv") for x in ("test", "train"))
test_df, train_df = (
    df.rename(columns={"NewsPaper": "newspaper", "News": "text", "Topic": "label"})
    for df in (test_df, train_df)
)
test_df, train_df = (df.drop(columns=["filename", "date", "newspaper"]) for df in (test_df, train_df))

In [83]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
from torchtext.legacy.data import TabularDataset

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [84]:
from konlpy.tag import Mecab
tokenizer = Mecab()

In [85]:
TEXT = data.Field(
    sequential = True,
    lower = True,
    use_vocab = True,
    tokenize = tokenizer.morphs,
)

In [86]:
LABEL = data.LabelField()
LABEL.build_vocab(train_df.label)
LABEL.vocab.stoi

defaultdict(None,
            {'IT/과학': 0,
             '경제': 1,
             '문화': 2,
             '미용/건강': 3,
             '사회': 4,
             '생활': 5,
             '스포츠': 6,
             '연예': 7,
             '정치': 8})

In [87]:
import re
prog = re.compile(r"</?p>")

In [88]:
# https://bab2min.tistory.com/544
stop_words = set(pd.read_table(f"{PATH}/stopwords-ko-100.txt", header=None)[0])
def preprocess(text):
    lst = TEXT.preprocess(re.sub(prog, "", text))
    return [word for word in lst if word not in stop_words]

for df in (test_df, train_df):
    df.text = df.text.apply(preprocess)

In [89]:
test_df

,label,text
0,IT/과학,"[인터넷, 게임, 족쇄, 에, 도내, 업체, 도, ‘, 덜덜, ’, 道, ㆍ업체, ..."
1,IT/과학,"[삼성전자, ,, un, 에, 삼, 성만, 의, 특화, 된, 생태, 보전, 활동, ..."
2,IT/과학,"[지자체, 의, ‘, 카카오, 채널, ’,, 도민, 관심, 부족, 으로, 폐쇄, 거..."
3,IT/과학,"[돈, 만, 잡아먹, 는, 홈페이지, ,, 과감, 게, ‘, 로그아웃, ’, 경기도..."
4,IT/과학,"[콘텐츠, 기업, 대상, ‘, 부천, 클러스터, ’, 입, 주사, 모집, 경기, 콘..."
...,...,...
1795,정치,"[안철수, ,, 독자, 출마, 로, …, 정치, 권, 배제, 채, 대권, 행보, 안..."
1796,정치,"[예, 산심, 의, 표류, …, 서민, ·, 저소득층, 울린다, 국회, 의, 예산,..."
1797,정치,"["", 퇴직금, 7000, 만, 은, 로비, 대가, "", 제기, ,, 커지, 는, ..."
1798,정치,"["", 국회의원, ㆍ장관, 겸직, 금지, 면, 임명권자, 인재, 풀, 좁, 아, 져..."


In [90]:
from torchtext.vocab import Vectors
vectors = Vectors(name=f"{PATH}/fasttext_morph_300.model")

In [91]:
TEXT.build_vocab(
    train_df.text,
    vectors=vectors,
    min_freq=3
)
vocab = TEXT.vocab

In [92]:
print(vocab["호주"])

1870


In [93]:
from torchtext.legacy.data import Dataset, Example

class DataFrameDataset(Dataset):
    def __init__(self, df, fields):
        super().__init__(
            [Example.fromlist(list(row), fields) for i, row in df.iterrows()],
            fields,
        )

In [94]:
fields = (
    ("label", LABEL),
    ("text", TEXT),
)
train_data, valid_data = DataFrameDataset(train_df, fields).split(split_ratio=0.8)
test_data = DataFrameDataset(test_df, fields)

In [95]:
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [96]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort = False,
)

In [97]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.convs = nn.ModuleList([
            nn.Conv2d(
                in_channels = 1, 
                out_channels = n_filters, 
                kernel_size = (fs, embedding_dim),
            ) for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):        
        #text = [sent len, batch size]
        text = text.permute(1, 0)
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [98]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [99]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 12,808,809 trainable parameters


In [100]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [101]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [102]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [103]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [28]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()

    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

In [31]:
MODEL_NAME = "newspaper-model-with-stops"

In [36]:
N_EPOCHS = 3
best_valid_loss = float("inf")

for epoch in range(1, N_EPOCHS + 1):
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f"{PATH}/{MODEL_NAME}.pt")

    print(f"Epoch: {10 + epoch:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%")
    print(f"\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%")

Epoch: 11 | Epoch Time: 1m 28s
	Train Loss: 0.341 | Train Acc: 89.08%
	 Val. Loss: 0.610 |  Val. Acc: 78.88%
Epoch: 12 | Epoch Time: 1m 31s
	Train Loss: 0.270 | Train Acc: 91.39%
	 Val. Loss: 0.626 |  Val. Acc: 78.88%
Epoch: 13 | Epoch Time: 1m 31s
	Train Loss: 0.215 | Train Acc: 93.43%
	 Val. Loss: 0.623 |  Val. Acc: 79.47%


In [104]:
model.load_state_dict(torch.load(f"{PATH}/{MODEL_NAME}.pt"))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f"Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%")

Test Loss: 0.575 | Test Acc: 80.06%


## User Input

####  뉴스 labels
    -  IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8

In [109]:
def predict_news(model, sentence, min_len=5):
    model.eval()
    tokenized = preprocess(sentence)
    if len(tokenized) < min_len:
        tokenized += ["<pad>"] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim=1)
    # print(preds)
    return max_preds.item()

In [110]:
lookup = ["IT/과학", "경제", "문화", "미용/사회", "사회", "생활", "스포츠", "연예", "정치"]
sentences = [
    "차진 식감과 부드러운 감촉을 모두 지닌 식빵, 결결이 찢어지는 크루아상, 둥글고 크게 구운 호밀빵 모두 모양새부터 알차고 단단했다. 디저트로 눈을 옮기면 국가 대표팀같이 뭐 하나 빼놓을 수 없는 케이크가 나란히 줄을 서 있었다.",
    "정부는 2012년 예산의 공고안과 배정계획을 1월3일 국무회의에서 의결하고 연초부터 바로 집행에 들어간다. 세계 경제의 불확실성이 높아지고 경기가 둔화할 가능성이 높은 만큼 조기 집행에 박차를 가할 예정이다",
    "국세청은 특히 서민생활에 피해를 주면서 폭리를 취하는 매점매석 농수산물 유통업체 등에 대한 추적조사를 강화하고, 지방청에 ‘민생침해 사업자 조사전담팀’을 꾸려 민생침해 탈세자에 대한 엄정 대응에 나설 계획이라고 밝혔다",
    "26일 제25회 부산국제영화제 갈라 프레젠테이션 부문 초청작 '스파이의 아내' 온라인 기자회견이 진행됐다. 작품을 연출한 구로사와 감독이 참석해 작품에 대한 이야기를 나눴다.",
    "70대 운전사가 몰던 25인승 어린이 통학버스가 주유소로 돌진해 차량 3대를 들이 받았다. 다행히 통학버스에 운전자 외에는 탑승자가 없어 큰 인명피해는 피했다. 운전자는 차량 결함을 주장하고 있으나, 경찰은 운전자 과실 여부도 조사 중이다.",
    "토트넘이 손흥민에게 주급 20만 파운드(약 3억원)-5년 재계약을 제안할 준비를 마쳤다.' 25일(한국시각) 영국 대중일간 더선의 헤드라인이다. 조제 무리뉴 토트넘 감독이 번리전을 앞두고 기자회견을 통해 구단에 토트넘에서의 손흥민의 장밋빛 미래를 확신하며 재계약을 요청한 직후 영국 현지 언론에선 손흥민 재계약 보도가 쏟아지고 있다.",
    "공연은 말 그대로 다채로움 그 자체였다. 발레극인지, 현대무용극인지, 전통극인지, 연극인지, 연극이면 다인극인지 1인극인지 모를 정도로 다양한 장르의 결합이 먼저 눈에 띈다.",
    "발이 아프면 걷는 자세가 나빠지고 자연스럽게 무릎, 골반, 허리에 이상이 생길 수 있다. 이를 예방하려면 평소 발바닥 근육을 스트레칭하고 강화하는 운동을 지속하는 게 중요하다.",
    "전기차 화재 사고가 연이어 발생하는 가운데 불타지 않는 SK이노베이션 배터리가 주목받는다. SK이노베이션의 배터리는 글로벌 배터리 업체 중 유일하게 단 한건의 화재도 일어나지 않았다. 이같은 비결이 자동차 안정성을 결정짓는 분리막 내재화에 있다는 평가가 나온다.",
]
# HUMAN PRED: 생활 - 경제 - 경제 - 문화 - 사회 - 스포츠 - 문화 - 미용/사회 - IT/과학
# 5, 1, 1, 2, 4, 6, 2, 3, 0
answers = [5, 8, 1, 2, 4, 6, 7, 3, 0]
cnt = 0
print("예상\t답안\t정답")
for i, sentence in enumerate(sentences):
    pred = predict_news(model, sentence)
    ans = answers[i]
    if pred == ans:
        cnt += 1
    print(f"{lookup[pred]}\t{lookup[ans]}\t{'O' if pred == ans else 'X'}")

print(f"\t\t{cnt}/{len(answers)}={cnt / len(answers) * 100:.0f}%")

예상	답안	정답
스포츠	생활	X
정치	정치	O
사회	경제	X
문화	문화	O
생활	사회	X
스포츠	스포츠	O
문화	연예	X
미용/사회	미용/사회	O
IT/과학	IT/과학	O
		5/9=56%
